In [71]:
import pandas as pd
import openpyxl

raw_excel = 'epitope_table_large.xlsx'

# Use pandas.read_excel() to read the Excel file into a DataFrame
df = pd.read_excel(raw_excel)

num_raw_entries = len(df)

print(df.columns)

Index(['Epitope ID - IEDB IRI', 'Epitope - Name',
       'Epitope - Starting Position', 'Epitope - Ending Position',
       'Epitope - Source Molecule', 'Epitope - Source Molecule IRI'],
      dtype='object')


In [67]:
df = df.dropna(subset=['Epitope - Starting Position', 'Epitope - Ending Position', 'Epitope - Source Molecule IRI'])

num_entries_1 = len(df)
print(num_raw_entries, num_entries_1)

999 964


In [68]:
valid_prefixes = ['http://www.ncbi.nlm.nih.gov/protein/', 'https://www.uniprot.org/uniprot/']

# Create a boolean mask to filter rows that start with a valid prefix
mask = df['Epitope - Source Molecule IRI'].str.startswith(tuple(valid_prefixes))

# Use the boolean mask to filter the DataFrame
df = df[mask]

num_entries_2 = len(df)
print(num_entries_2)
print(df.head)

963
<bound method NDFrame.head of                   Epitope ID - IEDB IRI        Epitope - Name  \
0       https://www.iedb.org/epitope/34                AAAGDK   
1       https://www.iedb.org/epitope/56               AAARTTS   
2       https://www.iedb.org/epitope/60      AAASAIQGNVTSIHSL   
3       https://www.iedb.org/epitope/76                AACRAA   
4       https://www.iedb.org/epitope/82        AADAILHTPGCVPC   
..                                  ...                   ...   
994  https://www.iedb.org/epitope/17997  FTKVCGAPPCVIGGAGNNTL   
995  https://www.iedb.org/epitope/17998          FTKWTWETKGDL   
996  https://www.iedb.org/epitope/18025         FTNKLKEKHTDLG   
997  https://www.iedb.org/epitope/18037        FTPGAKQNIQLINT   
998  https://www.iedb.org/epitope/18039               FTPGPSQ   

     Epitope - Starting Position  Epitope - Ending Position  \
0                            7.0                       12.0   
1                            7.0                       13.0

In [69]:
import requests as r
from Bio import SeqIO
from io import StringIO
from Bio import Entrez
Entrez.email = "ambri@princeton.edu"  # Set your email address
df['Epitope ID'] = 0

# Function to retrieve sequence from URL
def get_sequence_from_url(prefix, cID):
    response = ''
    
    if prefix == "http://www.ncbi.nlm.nih.gov/protein/":
        handle = Entrez.efetch(db="protein", id=cID,  rettype="fasta")
        fasta_sequence = handle.read()
        handle.close()
        # Skip the first header line
        lines = fasta_sequence.split('\n')
        sequence_lines = lines[1:]
        sequence_text = ''.join(sequence_lines)
        if len(sequence_text) > 1024:
                return 'TOO LONG'
        return sequence_text
    elif prefix == "https://www.uniprot.org/uniprot/":
        searchUrl ='http://www.uniprot.org/uniprot/' + cID + ".fasta"
     
        response = r.get(searchUrl)
      
        if response.status_code == 200:
            fasta_sequence = response.text
            # Skip the first header line
            lines = fasta_sequence.split('\n')
            sequence_lines = lines[1:]
            sequence_text = ''.join(sequence_lines)
            if len(sequence_text) > 1024:
                return 'TOO LONG'
            return sequence_text
        else:
            print("Failed to retrieve the FASTA sequence.")

# Iterate through the filtered DataFrame
for index, row in df.iterrows():
    url = row['Epitope - Source Molecule IRI']
    
    # Check if the URL starts with one of the valid prefixes
    for prefix in valid_prefixes:
        if url[:len(prefix)] == prefix:
            # Extract the code snippet from the URL
            cID = url[len(prefix):]
           

            sequence = get_sequence_from_url(prefix, cID)
        
            if sequence == 'TOO LONG':
                df.drop(index, inplace=True)
            else:
                # Replace the URL in 'Col2' with the retrieved sequence
                df.at[index, 'Epitope - Source Molecule IRI'] = sequence
                df.at[index, 'Epitope ID'] = cID
            break
    print(index, sequence)

/var/folders/zm/hzzkqkln6p55nm0jkx5xzfsh0000gn/T/ipykernel_79804/40867623.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'AAP88022.1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Epitope ID'] = cID


0 PPPFGQAAAGDKPSPFGQAAAGDKPPPFGQAAAGDKPSPFGQAAAGDKPPPFGQAAAGDKPSPFGQAAAGGKPPPFGQAAAGDKPSPFGQAAAGDKPPPFGQAAAGDKPSPFGQAAAGDKPPPFGQAAAGDKPPPFGQAAEGDKPPPFGQAAAGDKPAPFGQAAEGDKPPPFGQAAAADKPSLFGQAAAGDKLSLFGQAAAGDKPPPFGQAAEGDKPPPFGQAAAGD
1 TYATGGAAARTTSGFASLFSPGSAQ
2 MTEQQWNFAGIEAAASAIQGNVTSIHSLLDEGKQSLTKLAAAWGGSGSEAYQGVQQKWDATATELNNALQNLARTISEAGQAMASTEGNVTGMFA
3 TOO LONG
4 TOO LONG
5 TOO LONG
6 MSTLQELQENITAHEQQLVTARQKLKDAEKAVEVDPDDVNKSTLQNRRAAVSTLETKLGELKRQLADLVAAQKLATKPVDPTGLEPDDHLKEKSSLRYGNVLDVNSIDLEEPSGQTADWKAIGAYILGFAIPIILKALYMLSTRGRQAVKDNKGTRIRFKDDSSFEEVNGIRKPKHLYVSMPTAQSTMKAEEITPGRFRTIACGLFPAQVKARNIISPVMGVIGFGFFVKDWMDRIEEFLAAECPFLPKPKVASEAFMSTNKMYFLNRQRQVNESKVQDIIDLIDHAETESATLFTEIATPHSVWVFACAPDRCPPTALYVAGVPELGAFFSILQDMRNTIMASKSVGTAEEKLKKKSAFYQSYLRRTQSMGIQLDQKIIILYMLSWGKEAVNHFHLGDDMDPELRQLAQSLIDTKVKEISNQEPLKL
7 TOO LONG
8 MRPRPILLLLLMFLPMLPAPPPGQPSGRRRGRRSGGSGGGFWGDRADSQPFAIPYIHPTNPFAPDVTAAAGAGPRVRQPARPLGSAWRDQAQRPAAASRRRPTTAGAAPLTAVAPAHDTPPVPDVDSRGAILRRQYNLSTSPLTSSVATGTNLVLYAAPLSPLLPLQ

KeyboardInterrupt: 

In [ ]:
df = df.dropna(subset=['Epitope ID'])
file_path = 'processed_large.csv'
df.to_csv(file_path) 